# 使用微调的方法让模型对新闻分类更加准确

在本实操手册中，开发者将使用ChatGLM3-6B base模型，对新闻分类数据集进行微调，并使用微调后的模型进行推理。
本操作手册使用公开数据集，数据集中包含了新闻标题和新闻关键词，开发者需要根据这些信息，将新闻分类到15个类别中的一个。
为了体现模型高效的学习能力，以及让用户更快的学习本手册，我们只使用了数据集中的一小部分数据，实际上，数据集中包含了超过40万条新闻数据。

## 硬件要求
本实践手册需要使用 FP16 精度的模型进行推理，因此，我们推荐使用至少 16GB 显存的 英伟达 GPU 来完成本实践手册。


首先，我们将原始的数据集格式转换为用于微调的`jsonl`格式，以方便进行微调。

In [1]:
import json

# 路径可以根据实际情况修改
input_file_path = 'data/toutiao_cat_data_example.txt'
output_file_path = 'data/toutiao_cat_data_example.jsonl'

# 提示词
prompt_prefix = """
你是一个专业的新闻专家，请根据我提供的新闻信息，包括新闻标题，新闻关键词等信息，你要对每一行新闻类别进行分类并告诉我结果，不要返回其他信息和多于的文字，这些类别是:
news_story
news_culture
news_sports
news_finance
news_house
news_car
news_edu
news_tech
news_military
news_travel
news_world
stock
news_agriculture
news_game
请选择其中一个类别并返回，你只要返回类别的名称，不要返回其他信息。让我们开始吧:
"""

# 分类代码和名称的映射
category_map = {
    "100": "news_story",
    "101": "news_culture",
    "102": "news_entertainment",
    "103": "news_sports",
    "104": "news_finance",
    "106": "news_house",
    "107": "news_car",
    "108": "news_edu",
    "109": "news_tech",
    "110": "news_military",
    "112": "news_travel",
    "113": "news_world",
    "114": "stock",
    "115": "news_agriculture",
    "116": "news_game"
}

def process_line(line):
    # 分割每行数据
    parts = line.strip().split('_!_')
    if len(parts) != 5:
        return None

    # 提取所需字段
    _, category_code, _, news_title, news_keywords = parts

    # 构造 JSON 对象
    news_title = news_title if news_title else "无"
    news_keywords = news_keywords if news_keywords else "无"
    json_obj = {
        "context": prompt_prefix + f"新闻标题: {news_title}\n 新闻关键词: {news_keywords}\n",
        "target": category_map.get(category_code, "无")
    }
    return json_obj

def convert_to_jsonl(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as infile, \
         open(output_path, 'w', encoding='utf-8') as outfile:
        for line in infile:
            json_obj = process_line(line)
            if json_obj:
                json_line = json.dumps(json_obj, ensure_ascii=False)
                outfile.write(json_line + '\n')

# 运行转换函数
convert_to_jsonl(input_file_path, output_file_path)

## 使用没有微调的模型进行推理
首先，我们先试用原本的模基座模型进行推理，并查看效果。

In [2]:
PROMPT = """
你是一个专业的新闻专家，请根据我提供的新闻信息，包括新闻标题，新闻关键词等信息，你要对每一行新闻类别进行分类并告诉我结果，不要返回其他信息和多于的文字，这些类别是:
news_story
news_culture
news_sports
news_finance
news_house
news_car
news_edu
news_tech
news_military
news_travel
news_world
stock
news_agriculture
news_game
请选择其中一个类别并返回，你只要返回类别的名称，不要返回其他信息。让我们开始吧:
新闻标题：华为手机扛下敌人子弹，是什么技术让其在战争中大放异彩？
新闻关键词: 华为手机
"""

In [3]:
from transformers import AutoModel, AutoTokenizer
import  torch
# 参数设置
model_path = "THUDM/chatglm3-6b-base"
tokenizer_path = model_path
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, load_in_8bit=False, trust_remote_code=True).to(device)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
max_new_tokens = 1024
temperature = 0.4
top_p = 0.9
inputs = tokenizer(PROMPT, return_tensors="pt").to(device)
response = model.generate(input_ids=inputs["input_ids"],max_new_tokens=max_new_tokens,temperature=temperature,top_p=top_p,do_sample=True)
response = response[0, inputs["input_ids"].shape[-1]:]
origin_answer = tokenizer.decode(response, skip_special_tokens=True)
origin_answer

'news_house'

In [5]:
del model
torch.cuda.empty_cache()

我们可以发现，模型并没有正确的对新闻进行分类。这可能是由模型训练阶段的数据导致的问题。那么，我们通过微调这个模型，能不能实现更好的效果呢？

## 使用官方的微调脚本进行微调

在完成数据的切割之后，我们需要按照官方提供好的方式进行微调。我们使用的模型为`chatglm3-6b-base`基座模型，该模型相对于Chat模型，更容易上手微调，且更符合本章节的应用场景。

我们将对应的参数设置好后，就可以直接执行下面的代码进行微调。该代码使用`Lora`方案进行微调，成本相较于全参微调大幅度降低。


In [6]:
!which python
import os
from datetime import datetime
import random

# 定义变量
lr = 2e-5
num_gpus = 1
lora_rank = 8
lora_alpha = 32
lora_dropout = 0.1
max_source_len = 512
max_target_len = 128
dev_batch_size = 1
grad_accumularion_steps = 2
max_step = 300
save_interval = 100
max_seq_len = 512
logging_steps=1

run_name = "news"
dataset_path = "data/toutiao_cat_data_example.jsonl"
datestr = datetime.now().strftime("%Y%m%d-%H%M%S")
output_dir = f"output/{run_name}-{datestr}-{lr}"
master_port = random.randint(10000, 65535)

os.makedirs(output_dir, exist_ok=True)
# 构建命令
command = f"""
torchrun --standalone --nnodes=1 --nproc_per_node={num_gpus} ../finetune_basemodel_demo/finetune.py \
    --train_format input-output \
    --train_file {dataset_path} \
    --lora_rank {lora_rank} \
    --lora_alpha {lora_alpha} \
    --lora_dropout {lora_dropout} \
    --max_seq_length {max_seq_len} \
    --preprocessing_num_workers 1 \
    --model_name_or_path {model_path} \
    --output_dir {output_dir} \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 2 \
    --max_steps {max_step} \
    --logging_steps {logging_steps} \
    --save_steps {save_interval} \
    --learning_rate {lr}
"""

# 在 Notebook 中执行命令
!{command}

[2023-11-24 13:44:13,160] torch.distributed.run: [WARNING] master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
11/24/2023 13:44:15 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
11/24/2023 13:44:15 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,


## 使用微调的模型进行推理预测
现在，我们已经完成了模型的微调，接下来，我们将使用微调后的模型进行推理。我们使用与微调时相同的提示词，并使用一些没有出现的模型效果来复现推理结果。

In [7]:
import torch
from transformers import AutoModel, AutoTokenizer
import os
from peft import get_peft_model, LoraConfig, TaskType

# 参数设置
lora_path = output_dir + "pytorch_model.bin"

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, load_in_8bit=False, trust_remote_code=True).to(device)

# LoRA 模型配置
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    target_modules=['query_key_value'],
    r=8, lora_alpha=32, lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
if os.path.exists(lora_path):
    model.load_state_dict(torch.load(lora_path), strict=False)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

我们使用同样的提示词进行推理。

In [8]:
inputs = tokenizer(PROMPT, return_tensors="pt").to(device)
response = model.generate(input_ids=inputs["input_ids"],max_new_tokens=max_new_tokens,temperature=temperature,top_p=top_p,do_sample=True)
response = response[0, inputs["input_ids"].shape[-1]:]
response = tokenizer.decode(response, skip_special_tokens=True)

In [9]:
response

'news_tech'

这一次，模型成功给出了理想的答案。我们结束实操训练，删除模型并释放显存。

In [10]:
del model
torch.cuda.empty_cache()

## 总结
在本实践手册中，我们让开发者体验了 `ChatGLM3-6B` 模型在经过微调前后在 `新闻标题分类` 任务中表现。
我们可以发现：

在具有混淆的新闻分类中，原始的模型可能受到了误导，不能有效的进行分类，而经过简单微调后的模型，已经具备了正确分类的能力。
因此，对于有更高要求的专业分类任务，我们可以使用微调的方式对模型进行简单微调，实现更好的任务完成效果。
